In [10]:
%%bash
pip install xgboost
pip install lightgbm
pip install mlxtend
pip install graphviz

In [59]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import max_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel

from sklearn.decomposition import PCA
from mlxtend.regressor import StackingCVRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.stats import uniform, randint
from scipy import stats
import xgboost
from xgboost import XGBRegressor
from xgboost import plot_importance

from lightgbm import LGBMRegressor
import graphviz
import warnings
warnings.filterwarnings('ignore')

In [12]:
import sys  
!{sys.executable} -m pip install --user missingno

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import missingno
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from scipy import stats
from sklearn.manifold import TSNE
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
import math

In [2]:
train_data = pd.read_csv('Case1/case1Data.txt', sep=", ", header=0,index_col=None)
train_data = train_data.replace(' NaN',np.nan)
DatawMissing =train_data.replace(np.nan, 'Missing')


/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [4]:
x_train_nan, x_test_nan, y_train, y_test = train_test_split(train_data.drop(columns='y'),train_data.iloc[:,0],test_size=0.2, random_state=42)
x_train_missing, x_test_missing, y_train, y_test = train_test_split(DatawMissing.drop(columns='y'),DatawMissing.iloc[:,0],test_size=0.2, random_state=42)


In [7]:
x_train_full_missing = DatawMissing.drop(columns='y')
y_train_full_missing = DatawMissing.iloc[:,0]

In [8]:
num_pipeline = Pipeline([
('std_scaler', StandardScaler()),
])
num_attribs = list(train_data[train_data.columns[1:-5]])
cat_attribs = ["C_1", "C_2","C_3","C_4","C_5"]
full_pipeline = ColumnTransformer([
("num", num_pipeline, num_attribs),
("cat",  OneHotEncoder(), cat_attribs), 
])

In [9]:
x_train_nan_prepared = full_pipeline.fit_transform(x_train_nan)
x_test_nan_prepared = full_pipeline.transform(x_test_nan)
x_train_missing_prepared = full_pipeline.fit_transform(x_train_missing)
x_test_missing_prepared = full_pipeline.transform(x_test_missing)
x_train_full_missing_prepared = full_pipeline.fit_transform(x_train_full_missing)

In [96]:
lasso_reg = Lasso()
lasso_reg.fit(x_train_missing_prepared, y_train)
y_pred = lasso_reg.predict(x_test_missing_prepared)
print(np.sqrt(mean_squared_error(y_test,y_pred)))
print(max_error(y_test,y_pred))
print(r2_score(y_test,y_pred))

38.70818975466528
96.95906876368696
0.7000475258192909


In [94]:
forest_reg = RandomForestRegressor(oob_score = True,random_state=42)
forest_reg.fit(x_train_missing_prepared, y_train)
y_pred = forest_reg.predict(x_test_missing_prepared)

print(np.sqrt(mean_squared_error(y_test,y_pred)))
print(max_error(y_test,y_pred))
print(r2_score(y_test,y_pred))
print(forest_reg.oob_score_)
important_features_dict = {}
for idx, val in enumerate(forest_reg.feature_importances_):
    important_features_dict[idx] = val

important_features_list = sorted(important_features_dict,
                                 key=important_features_dict.get,
                                 reverse=True)

print('9 most important features:',important_features_list[:9])

37.823132031926825
85.78877058999994
0.7136074585788774
0.6341730518170546
9 most important features: [72, 41, 51, 94, 50, 66, 71, 89, 13]


In [100]:
xgb_reg = XGBRegressor(learning_rate=0.01,n_estimators=3460,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     objective='reg:linear', nthread=-1,
                                     scale_pos_weight=1, seed=42,
                                     reg_alpha=0.00006,verbosity=0)
xgb_reg.fit(x_train_nan_prepared, y_train)
y_pred = xgb_reg.predict(x_test_nan_prepared)

print(np.sqrt(mean_squared_error(y_test,y_pred)))
print(max_error(y_test,y_pred))
print(r2_score(y_test,y_pred))
important_features_dict = {}
for idx, val in enumerate(xgb_reg.feature_importances_):
    important_features_dict[idx] = val

important_features_list = sorted(important_features_dict,
                                 key=important_features_dict.get,
                                 reverse=True)

print('9 most important features:',important_features_list[:9])

36.52486301873651
78.45082506933593
0.7329307310950741
9 most important features: [72, 41, 51, 118, 94, 89, 105, 43, 83]


## Cross Validation 

In [38]:
kfold = KFold(n_splits=5, random_state=42,shuffle=True)

In [77]:
results = cross_val_score(xgb_reg, x_train_missing_prepared, y_train, scoring="neg_mean_squared_error",cv=kfold)
#print("XGBoost Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
print("XGBoost RMSE:", (np.sqrt(-results.mean()), results.std()))

[17:34:47] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[17:34:49] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[17:34:51] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[17:34:53] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[17:34:55] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
XGBoost RMSE: (49.09702574262583, 1805.294474347253)


In [83]:
results = cross_val_score(forest_reg, x_train_missing_prepared, y_train,  scoring="neg_mean_squared_error",cv=kfold)
print("Random forest RMSE:", (np.sqrt(-results.mean()), results.std()))

Random forest RMSE: (54.2983623314413, 1981.8450117429363)


In [79]:
results = cross_val_score(lasso_reg, x_train_missing_prepared, y_train,  scoring="neg_mean_squared_error", cv=kfold)
print("Lasso  RMSE:", (np.sqrt(-results.mean()), results.std()))

Lasso  RMSE: (35.442059505997136, 256.5820162033372)


### GridSearchCV for all model

In [111]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'bootstrap': [True],
    'max_depth': [40, 50,60],
    'max_features': [5],
    'min_samples_leaf': [1],
    'min_samples_split': [1,2,4],
    'n_estimators': [40,60]
}
grid_search = GridSearchCV(estimator = forest_reg, param_grid = param_grid, 
                          cv =kfold, n_jobs = -1, verbose = 1,scoring='explained_variance')
grid_search.fit(x_train_missing_prepared, y_train)
grid_search.best_params_
final_forest_reg = grid_search.best_estimator_

Fitting 5 folds for each of 18 candidates, totalling 90 fits


In [88]:
results = cross_val_score(final_forest_reg, x_test_missing_prepared, y_test, cv=kfold)
print("Random forest RMSE:", (np.sqrt(-results.mean()), results.std()))

Random forest RMSE: (0.15411820237512763, 0.12431758321129517)


In [112]:
print(grid_search.best_params_)
grid_search.best_score_

{'bootstrap': True, 'max_depth': 40, 'max_features': 5, 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 60}


0.33139546621345817

## CV with Feature selection

In [54]:
for i in range(3,12):
    selector = SelectFromModel(estimator=forest_reg,max_features=i,prefit=True)
    fsXTRAIN=selector.transform(x_train_full_missing_prepared)
    results = cross_val_score(forest_reg, fsXTRAIN, y_train_full_missing, cv=kfold)
    print("for the",i,"most important features:")
    print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

for the 3 most important features:
Accuracy: 66.74% (9.82%)
for the 4 most important features:
Accuracy: 66.54% (10.43%)
for the 5 most important features:
Accuracy: 79.53% (6.27%)
for the 6 most important features:
Accuracy: 78.34% (6.76%)
for the 7 most important features:
Accuracy: 77.23% (7.47%)
for the 8 most important features:
Accuracy: 75.36% (8.76%)
for the 9 most important features:
Accuracy: 75.06% (8.78%)
for the 10 most important features:
Accuracy: 74.67% (9.25%)
for the 11 most important features:
Accuracy: 73.89% (8.10%)


In [105]:
for i in range(1,18):
    selector = SelectFromModel(estimator=lasso_reg,max_features=i,prefit=True)
    fsXTRAIN=selector.transform(x_train_full_missing_prepared)
    results = cross_val_score(lasso_reg, fsXTRAIN, y_train_full_missing, cv=kfold)
    print("for the",i,"most important features:")
    print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

for the 1 most important features:
Accuracy: 46.51% (9.80%)
for the 2 most important features:
Accuracy: 67.03% (4.48%)
for the 3 most important features:
Accuracy: 81.92% (6.85%)
for the 4 most important features:
Accuracy: 83.03% (6.93%)
for the 5 most important features:
Accuracy: 83.43% (6.97%)
for the 6 most important features:
Accuracy: 84.81% (5.86%)
for the 7 most important features:
Accuracy: 85.37% (5.55%)
for the 8 most important features:
Accuracy: 85.24% (5.62%)
for the 9 most important features:
Accuracy: 85.18% (5.72%)
for the 10 most important features:
Accuracy: 84.97% (6.10%)
for the 11 most important features:
Accuracy: 85.20% (5.54%)
for the 12 most important features:
Accuracy: 85.06% (5.76%)
for the 13 most important features:
Accuracy: 84.56% (6.30%)
for the 14 most important features:
Accuracy: 84.16% (6.81%)
for the 15 most important features:
Accuracy: 84.13% (6.86%)
for the 16 most important features:
Accuracy: 83.99% (6.93%)
for the 17 most important feature

In [103]:
for i in range(2,16):
    selector = SelectFromModel(estimator=xgb_reg,max_features=i,prefit=True)
    fsXTRAIN=selector.transform(x_train_full_missing_prepared)
    results = cross_val_score(xgb_reg, fsXTRAIN, y_train_full_missing, cv=kfold)
    print("for the",i,"most important features:")
    print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

for the 2 most important features:
Accuracy: 48.08% (23.22%)
for the 3 most important features:
Accuracy: 64.41% (12.38%)
for the 4 most important features:
Accuracy: 61.70% (12.86%)
for the 5 most important features:
Accuracy: 64.31% (12.23%)
for the 6 most important features:
Accuracy: 64.18% (14.74%)
for the 7 most important features:
Accuracy: 63.46% (14.27%)
for the 8 most important features:
Accuracy: 60.88% (15.75%)
for the 9 most important features:
Accuracy: 61.22% (12.02%)
for the 10 most important features:
Accuracy: 82.89% (5.31%)
for the 11 most important features:
Accuracy: 81.74% (5.49%)
for the 12 most important features:
Accuracy: 81.33% (5.36%)
for the 13 most important features:
Accuracy: 81.44% (5.95%)
for the 14 most important features:
Accuracy: 78.96% (6.58%)
for the 15 most important features:
Accuracy: 79.87% (6.53%)
